In [1]:
from neo4j import GraphDatabase

class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response


conn = Neo4jConnection(uri="neo4j://54.75.63.152:7687", 
                       user="neo4j",              
                       pwd="password")


In [4]:
conn.query('CREATE (:Persona{nombre:"Ivan"})')


[]

In [2]:
p=conn.query('MATCH (persona:Persona) RETURN persona.nombre, persona.nacimiento')


In [3]:
p

[<Record persona.nombre='Michael Sheen' persona.nacimiento=1969.0>,
 <Record persona.nombre='Jack Nicholson' persona.nacimiento=1937.0>,
 <Record persona.nombre='Nathan Lane' persona.nacimiento=1956.0>,
 <Record persona.nombre='Philip Seymour Hoffman' persona.nacimiento=1967.0>,
 <Record persona.nombre='Noah Wyle' persona.nacimiento=1971.0>,
 <Record persona.nombre="Rosie O'Donnell" persona.nacimiento=1962.0>,
 <Record persona.nombre='Greg Kinnear' persona.nacimiento=1963.0>,
 <Record persona.nombre='Susan Sarandon' persona.nacimiento=1946.0>,
 <Record persona.nombre='Takeshi Kitano' persona.nacimiento=1947.0>,
 <Record persona.nombre='Gary Sinise' persona.nacimiento=1955.0>,
 <Record persona.nombre='John Goodman' persona.nacimiento=1960.0>,
 <Record persona.nombre='Christina Ricci' persona.nacimiento=1980.0>,
 <Record persona.nombre='Jay Mohr' persona.nacimiento=1970.0>,
 <Record persona.nombre='Ben Miles' persona.nacimiento=1967.0>,
 <Record persona.nombre='Carrie Fisher' persona.nac

In [7]:
d=dict(p)

In [8]:
print(d)

{'Michael Sheen': 1969.0, 'Jack Nicholson': 1937.0, 'Nathan Lane': 1956.0, 'Philip Seymour Hoffman': 1967.0, 'Noah Wyle': 1971.0, "Rosie O'Donnell": 1962.0, 'Greg Kinnear': 1963.0, 'Susan Sarandon': 1946.0, 'Takeshi Kitano': 1947.0, 'Gary Sinise': 1955.0, 'John Goodman': 1960.0, 'Christina Ricci': 1980.0, 'Jay Mohr': 1970.0, 'Ben Miles': 1967.0, 'Carrie Fisher': 1956.0, 'Christopher Guest': 1948.0, 'Lori Petty': 1963.0, 'Carrie-Anne Moss': 1967.0, 'Laurence Fishburne': 1961.0, 'Zach Grenier': 1954.0, 'John Hurt': 1940.0, 'Matthew Fox': 1966.0, 'Kelly Preston': 1962.0, 'Brooke Langton': 1970.0, 'Ian McKellen': 1939.0, 'Sam Rockwell': 1968.0, 'Renee Zellweger': 1969.0, 'J.T. Walsh': 1943.0, "Jerry O'Connell": 1974.0, 'Ethan Hawke': 1970.0, 'Ice-T': 1958.0, 'Bonnie Hunt': 1961.0, 'Richard Harris': 1930.0, 'Julia Roberts': 1967.0, 'Danny DeVito': 1944.0, 'Audrey Tautou': 1976.0, 'Charlize Theron': 1975.0, 'Frank Langella': 1938.0, 'Kevin Pollak': 1957.0, 'Steve Zahn': 1967.0, 'Aaron Sorkin

In [11]:
from pandas import DataFrame
DataFrame(p)

,0,1
0,Michael Sheen,1969.0
1,Jack Nicholson,1937.0
2,Nathan Lane,1956.0
3,Philip Seymour Hoffman,1967.0
4,Noah Wyle,1971.0
...,...,...
97,Dina Meyer,1968.0
98,James Marshall,1967.0
99,Clint Eastwood,1930.0
100,Diane Keaton,1946.0


In [29]:
file = "https://data.neo4j.com/intro/movies/movies.csv"

metadata  = []

lines = 100000    # 100k for testing

with open(file, 'r') as f:
    
    for line in tqdm(f):
        metadata.append(json.loads(line))
        lines -= 1
        if lines == 0: break
            
df = pd.DataFrame(metadata)


FileNotFoundError: [Errno 2] No such file or directory: 'https://data.neo4j.com/intro/movies/movies.csv'

In [30]:
import pandas as pd
import io
import requests
url="https://data.neo4j.com/intro/movies/movies.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))
c

,title,released,tagline
0,The Matrix,1999,Welcome to the Real World
1,Something's Gotta Give,1975,NaN
2,Ninja Assassin,2009,Prepare to enter a secret world of assassins
3,The Matrix Reloaded,2003,Free your mind
4,Stand By Me,1995,"For some, it's the last real taste of innocenc..."
5,The Birdcage,1996,Come as you are
6,Sleepless in Seattle,1993,"What if someone you never met, someone you nev..."
7,Charlie Wilson's War,2007,A stiff drink. A little mascara. A lot of nerv...
8,The Polar Express,2004,This Holiday Season… Believe
9,That Thing You Do,1996,In every life there comes a time when that thi...


In [32]:
query = '''
            UNWIND $rows AS row
            MERGE (c:Pelicula {title: row.title})
            RETURN count(*) as total
            '''
v= conn.query(query, parameters = {'rows':c.to_dict('records')})

In [33]:
v

[<Record total=38>]

In [37]:
c.to_dict('records')

[{'title': 'The Matrix',
  'released': 1999,
  'tagline': 'Welcome to the Real World'},
 {'title': "Something's Gotta Give", 'released': 1975, 'tagline': nan},
 {'title': 'Ninja Assassin',
  'released': 2009,
  'tagline': 'Prepare to enter a secret world of assassins'},
 {'title': 'The Matrix Reloaded',
  'released': 2003,
  'tagline': 'Free your mind'},
 {'title': 'Stand By Me',
  'released': 1995,
  'tagline': "For some, it's the last real taste of innocence, and the first real taste of life. But for everyone, it's the time that memories are made of."},
 {'title': 'The Birdcage', 'released': 1996, 'tagline': 'Come as you are'},
 {'title': 'Sleepless in Seattle',
  'released': 1993,
  'tagline': 'What if someone you never met, someone you never saw, someone you never knew was the only someone for you?'},
 {'title': "Charlie Wilson's War",
  'released': 2007,
  'tagline': "A stiff drink. A little mascara. A lot of nerve. Who said they couldn't bring down the Soviet empire."},
 {'title'